# ディープラーニングとは

層を深くしたニューラルネットワーク。CNNやパラメータの最適化手法などは重要な技術。<br>
以下は、学習済みのパタメータを利用して実装したもの。

In [6]:
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *


class DeepConvNet:
    """認識率99%以上の高精度なConvNet

    ネットワーク構成は下記の通り
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 重みの初期化===========
        # 各層のニューロンひとつあたりが、前層のニューロンといくつのつながりがあるか（TODO:自動で計算する）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        weight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLUを使う場合に推奨される初期値
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # レイヤの生成===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [7]:
#学習済みデータの精度を算出
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()

network.load_params(file_name="deep_convnet_params.pkl")

network.accuracy(x_test,t_test)

0.9935

これまでのネットワークよりも高い精度である。

# 有名なネットワーク

ImageNetは、100万枚を超える画像のデータセット。<br>
ILSVRCのコンペティションの「クラス分類」では、ImageNetのデータを、1000クラスに分類する。<br>

![画像を表示できません。](image.png)
2012年を境に、ディープラーニングによる手法が常にトップに立っている。<br>
上のうちVGG、GoogLeNet、ResNetのモデルを見る。

## VGG

VGGはシンプルな構造で応用性が高いため、VGGベースのネットワークは多く利用される。

VGGは基本的なCNNの構造で、全部で16または19層まで重ねて層を深くしている。<br>
3×3の小さなフィルターによる畳み込み層を連続して行なっていることが特徴。

## GoogLeNet

![](image2.2.png)

GoogLeNetは縦方向の深さだけでなく、横方向にも広がりを持つ**インセプション構造**をベースとしていることが特徴。<br>
サイズの異なるフィルター及びプーリングを複数適用し、結果を融合する構造をひとつのブロックとしている。<br><br>
またフィルターサイズ1×1の畳み込み層を多く使用し、チャンネル方向のサイズを減らすことで、パラメータ削減や処理の高速化をしている。

## ResNet

![](image2.3.png)

ディープラーニングの学習において、層を深くしすぎると学習がうまくいかないことが多々あった。<br>
ResNeの特徴は、この問題の回避のために「スキップ構造」を導入し、層を深くできるようにしたこと。<br>
逆伝播の際も、信号が減衰することなく伝わっていくため、勾配消失の問題を回避することができ、高い精度を達成した。<br>

# 精度向上の手法

「What is the class of this image ?」のWebサイトには、論文で発表された手法の認識精度がランキング形式で紹介されている。<br>
ほとんどの手法で、**CNN**が用いられている。<br>
ランキング上位の手法には、ネットワークの精度を高めるために参考にできる技術がある。

## アンサンブル学習

## 学習係数の減衰（learning rate decay）

## データ拡張（Data Augmentation）

![画像を表示できません。](image2.png)
回転や移動による「変形」、画像の一部を切り出す「crop処理」、左右を反転させる「flip処理」の他、画像の輝度の変化や、拡大・縮小のスケール変化をつけるなどし、訓練画像を増やすことは、認識精度の向上に有効。

## 層を深くすること

新たなコンピュータの技術や、ビッグデータなどの新たな環境によって、層を深くし正しく学習することができるようになった。<br>
ILSBRCなどのコンペティションの上位を占める手法の多くはディープラーニングによる手法。<br>
層を深くすると認識精度が向上する可能性がある。以下はそのメリット。

### パラメータ削減

![画像を表示できません。](image3.png)
層を深くすると、より少ないパラメータで同レベルの表現力を達成できる。<br>
層を深くすればするほど、パラメータ数の削減の割合は高くなる（7×7と3×3×3→削減率0.45、9×9と3×3×4→削減率0.56）。<br>
重み$m$×$m$、層の深さを$n$のモデルのパラメータ数の削減率は$\dfrac{m^{2}n^{2}-m^{2}n-2mn^{2}+n^{2}+2nm-2n+1}{m^{2}n^{2}-2mn^{2}+n^{2}+2nm-2n+1}$で表され、

### 表現力の向上

小さなフィルターを重ねてネットワークを深くすることで、**受容野**（ニューロンに変化を生じさせる局所的な空間領域）を広くカバーできる。<br>
また層を深くすると、畳み込み層の間に「非線形」の活性化関数を挟むことができ、ネットワークがより複雑な表現をできるようになる。

### 学習の階層的な分解

CNNの畳み込み層が階層的に情報を抽出している。<br>
例えば画像認識の場合、最初の層はエッジだけを学習し、次以降の層はそのエッジの情報を使ってテクスチャや物体のパーツなど、段階的に複雑な処理ができるようになる。<br>
複雑な問題を、解きやすいシンプルな問題に分解すことができ、効率的に学習ができる。

# 高速化の手法

ディープラーニングにおいて最も時間のかかる部分は、畳み込み層の計算である。<br>
つまり、積和演算の高速化が重要な課題である。

## GPUによる高速化

連続的で複雑な計算を得意とするCPUに対し、GPUは大量の並列的な計算を得意とする。<br>
大量の積和演算を**GPUコンピューティング**によって高速に行うことができる。<br><br>
NVIDIA社提供のCUDAという統合開発環境がディープラーニングのフレームワークで作られているため、ディープラーニングにおいてはNVIDIA提供のGPUから恩恵を受けられる。<br>
CUDA上で動作するcuDNNにはディープラーニング用に最適化された関数などが実装されている。

## 分散学習

複数のマシンを使用し、使用するGPUを増やすことで学習速度向上を目指すもの。<br>
分散学習にいおいては「マシン間での通信」、「データの同期」など多くの問題がある。<br>
TensorFlowやCNTKなどの優れたフレームワークを利用することができる。

## 演算精度のビット削減

ニューラルネットワークのロバスト性により、ディープラーニングでは数値精度のビット数をそこまで必要としないことが分かっている。<br>
大量の数値処理をするディープラーニングにおいては、メモリ容量やバス帯域についても考慮する必要があるため、数値を少し劣化させる。<br><br>
以下のように、Numpyの半精度浮動小数点数を使っても認識精度はあまり低下しない。

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from deep_convnet import DeepConvNet
from dataset.mnist import load_mnist


(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()
network.load_params("deep_convnet_params.pkl")

sampled = 10000 # 高速化のため
x_test = x_test[:sampled]
t_test = t_test[:sampled]

print("caluculate accuracy (float64) ... ")
print(network.accuracy(x_test, t_test))

# float16に型変換
x_test = x_test.astype(np.float16)
for param in network.params.values():
    param[...] = param.astype(np.float16)

print("caluculate accuracy (float16) ... ")
print(network.accuracy(x_test, t_test))

caluculate accuracy (float64) ... 
0.9935
caluculate accuracy (float16) ... 
0.9935


Pascalアーキテクチャ（NVIDIAの次世代GPU）では16ビットの半精度浮動小数点数（half float）の演算もサポートされるため、今後は半精度浮動小数点数が標準的に用いられると考えられる。<br><br>
最近では、重みや中間データを1ビットで表グェンする「**Binarized Neural Networks**」という手法が提案されている。

# 実用例と展望

ディープラーニングでは、手書き数字認識などのクラス分類に代表される**物体認識**をはじめ、画像や音声、自然言語など、多くの分野に対して優れた性能を発揮する。

## 物体検出

物体検出は、画像中から物体の位置と物体の種類を特定する問題であり、物体認識よりも複雑。<br>
CNNベースの手法がいくつか提案されているが、その中でも有名な**R-CNN**の処理フローを以下に示す。

重要なポイントは、「2. 候補領域抽出」と「3. CNN特徴の計算」。<br>
R-CNNの論文では、候補領域抽出で**Selective Search**と呼ばれる手法が使われている。<br><br>
最近では、候補領域抽出もCNNで行う**Faster R-CNN**が提案されている。全てをひとつのCNNで行うため、高速な処理が可能。

## セグメンテーション

![](image5.1.png)
引用：Everingham, Mark, and John Winn. "The PASCAL visual object classes challenge 2012 (VOC2012) development kit." Pattern Anal. Stat. Model. Comput. Learn., Tech. Rep 2007 (2012): 1-45.

セグメンテーションは、ピクセルレベルでクラス分類を行う問題。<br>
ピクセルごとに推論処理を行うと多くの無駄な計算が発生してしまうが、**FCN**はこれを改善する。<br>

![](image5.2.1.png)
引用：Long, Jonathan, Evan Shelhamer, and Trevor Darrell. "Fully convolutional networks for semantic segmentation." Proceedings of the IEEE conference on computer vision and pattern recognition. 2015.<br><br>
一般的なCNNは全結合層を含むのに対し、FCNはそれを同じ働きをする畳み込み層に置き換える。

## 画像キャプション生成

![](image5.3.png)
引用：Vinyals, Oriol, et al. "Show and tell: A neural image caption generator." Proceedings of the IEEE conference on computer vision and pattern recognition. 2015.

コンピュータビジョンと自然言語を融合させた研究。<br>
**NIC**と呼ばれる代表的なモデルは、層が深いCNNと、自然言語を扱うための**RNN**を組み合わせた**マルチモーダル処理**が行われる。<br>
具体的には、CNNによって画像から情報を抽出し、RNNはそれを初期値として再帰的にテキストを生成していく。<br>
RNNは、再起的なつながりを持つネットワークであり、自然言語や時系列データなど持続性があるデータに対して用いられることが多い。

## 画像スタイル変換

![](image5.4.png)
引用：neural-style "Touch implementation of neural style algorithm"

上のように、描画スタイルを指定する「スタイル画像」と元画像である「コンテンツ画像」を入力する。<br>
ネットワークの中間データが「コンテンツ画像」の中間データに近づくように学習することで、入力画像をコンテンツ画像の形状に似せる。<br>
「スタイル画像」からスタイルを吸収するために、スタイル行列という概念を導入し、このズレを小さくするように学習することで、スタイルを似せることができる。

## 画像生成

**DCGAN**はリアルな画像を新たに生成するもの。<br>
テーマに合う画像を大量に学習し、Generator（画像を生成）とDiscriminator（生成されたものか本物かを判断）の2つのニューラルネットワークが競うように成長していく**GAN**と呼ばれる技術が用いられている。<br><br>
これは、データと教師ラベルが対になって与えられる**教師あり学習**ではなく、教師データが与えられない**教師なし学習**である。

## 自動運転

さまざまな環境でもロバストに走行領域を正しく認識できるようになる必要がある。<br>
**SegNet**と呼ばれるCNNベースのネットワークでは、高精度に走路環境を認識し、入力画像に対しセグメンテーション（ピクセルレベルの判定）を行う。

## Deep Q-Network（強化学習）

強化学習は、試行錯誤の過程から自立的に学習させようとするもの。<br>
基本的な枠組みは、エージェントが環境に応じて行動を選択し、行動によって報酬を得て、できるだけ多くの報酬を得られるように行動方針を決めるということ。<br><br>
DQNはQ学習と呼ばれる最適行動価値関数を使用した強化学習のアルゴリズムをベースとする。<br>
その関数を近似するためにCNNを用いるのがDQN。<br><br>
囲碁のAlphaGoをはじめ、ディープラーニングや強化学習の性能が人間を上回ることが多くなってきた。